## 章节前言
自编码器是能够在无监督（即，训练集是未标记）的情况下学习输入数据的紧密表征（叫做潜在表征或编码）的人工神经网络。这些编码通常具有比输入数据低得多的维度，使得自编码器对降维有用（参见第 8 章）。自编码器还可以作为强大的特征检测器，它们可以用于无监督的深度神经网络预训练（正如我们在第 11 章中讨论过的）。最后，一些自编码器是生成式模型：他们能够随机生成与训练数据非常相似的新数据。例如，您可以在脸图片上训练自编码器，然后可以生成新脸。但是生成出来的图片通常是模糊且不够真实。

相反，用对抗生成网络（GAN）生成的人脸可以非常逼真，甚至让人认为他们是真实存在的人。你可以去这个网址，这是用 StyleGAN 生成的人脸，自己判断一下（还可以去这里，看看 GAN 生成的卧室图片），GAN 现在广泛用于超清图片涂色，图片编辑，将草图变为照片，增强数据集，生成其它类型的数据（比如文本、音频、时间序列），找出其它模型的缺点并强化，等等。

自编码器和 GAN 都是无监督的，都可以学习紧密表征，都可以用作生成模型，有许多相似的应用，但原理非常不同：

自编码器是通过学习，将输入复制到输出。听起来很简单，但内部结构会使其相当困难。例如，你可以限制潜在表征的大小，或者可以给输入添加噪音，训练模型恢复原始输入。这些限制阻止自编码器直接将输入复制到输出，可以强迫模型学习数据的高效表征。总而言之，编码是自编码器在一些限制下学习恒等函数的副产品。

GAN 包括两个神经网络：一个生成器尝试生成和训练数据相似的数据，一个判别器来区分真实数据和假数据。特别之处在于，生成器和判别器在训练过程中彼此竞争：生成器就像一个制造伪钞的罪犯，而判别器就像警察一样，要把真钱挑出来。对抗训练（训练竞争神经网络），被认为是近几年的一大进展。在 2016 年，Yann LeCun 甚至说 GAN 是过去 10 年机器学习领域最有趣的发明。

本章中，我们先探究自编码器的工作原理，如何做降维、特征提取、无监督预训练将、如何用作生成式模型。然后过渡到 GAN。先用 GAN 生成假图片，可以看到训练很困难。会讨论对抗训练的主要难点，以及一些解决方法。先从自编码器开始。

## 有效的数据表征
以下哪一个数字序列更容易记忆？

40, 27, 25, 36, 81, 57, 10, 73, 19, 68
50, 48, 46, 44, 42, 40, 38, 36, 34, 32, 30, 28, 26, 24, 22, 20, 18, 16, 14

乍一看，第一个序列似乎应该更容易，因为它要短得多。 但是，如果仔细观察第二个序列，就会发现它是从 50 到 14 的偶数。一旦你注意到这个规律，第二个序列比第一个更容易记忆，因为你只需要记住规律就成，开始的数字和结尾的数字。请注意，如果您可以快速轻松地记住非常长的序列，则不会在意第二个序列中存在的规律。 只要记住每一个数字，就够了。 事实上，很难记住长序列，因此识别规律非常有用，并且希望能够澄清为什么在训练过程中限制自编码器会促使它发现并利用数据中的规律。

记忆、感知和模式匹配之间的关系在 20 世纪 70 年代早期由 William Chase 和 Herbert Simon 研究。 他们观察到，专业棋手能够通过观看棋盘 5 秒钟就能记住所有棋子的位置，这是大多数人认为不可能完成的任务。 然而，只有当这些棋子被放置在现实位置（来自实际比赛）时才是这种情况，而不是随机放置棋子。 国际象棋专业棋手没有比你更好的记忆，他们只是更容易看到国际象棋的规律，这要归功于他们的比赛经验。 观察规律有助于他们有效地存储信息。

就像这个记忆实验中的象棋棋手一样，一个自编码器会查看输入信息，将它们转换为高效的潜在表征，然后输出一些（希望）看起来非常接近输入的东西。 自编码器总是由两部分组成：将输入转换为潜在表征的编码器（或识别网络），然后是将潜在表征转换为输出的解码器（或生成网络）（见图 17-1）。

<img src='
https://hands1ml.apachecn.org/img/133f15a30a9de1a24f0a24873dd09824.png
'>
<br>

如你所见，自编码器通常具有与多层感知器（MLP，请参阅第 10 章）相同的体系结构，但输出层中的神经元数量必须等于输入数量。 在这个例子中，只有一个由两个神经元（编码器）组成的隐藏层和一个由三个神经元（解码器）组成的输出层。由于自编码器试图重构输入，所以输出通常被称为重建，并且损失函数包含重建损失，当重建与输入不同时，重建损失会对模型进行惩罚。

由于内部表征具有比输入数据更低的维度（它是 2D 而不是 3D），所以自编码器被认为是不完整的。 不完整的自编码器不能简单地将其输入复制到编码，但它必须找到一种方法来输出其输入的副本。 它被迫学习输入数据中最重要的特征（并删除不重要的特征）。

我们来看看如何实现一个非常简单的不完整的自编码器，以降低维度。

## 用不完整的线性自编码器来做 PCA
如果自编码器仅使用线性激活并且损失函数是均方误差（MSE），最终其实是做了主成分分析（参见第 8 章）。

以下代码创建了一个简单的线性自编码器，以在 3D 数据集上执行 PCA，并将其投影到 2D：

In [ ]:
from tensorflow import keras

encoder = keras.models.Sequential([keras.layers.Dense(2, input_shape=[3])])
decoder = keras.models.Sequential([keras.layers.Dense(3, input_shape=[2])])
autoencoder = keras.models.Sequential([encoder, decoder])

autoencoder.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=0.1))

这段代码与我们在前面章节中创建的所有 MLP 没有什么大不同。只有以下几点要注意：

自编码器由两部分组成：编码器和解码器。两者都是常规的Sequential模型，每个含有一个紧密层，自编码器是一个编码器和解码器连起来的Sequential模型（模型可以用作其它模型中的层）。

自编码器的输出等于输入。

简单 PCA 不需要激活函数（即，所有神经元是线性的），且损失函数是 MSE。后面会看到更复杂的自编码器。

现在用生成出来的 3D 数据集训练模型，并用模型编码数据集（即将其投影到 2D）：

In [ ]:
history = autoencoder.fit(X_train, X_train, epochs=20)
codings = encoder.predict(X_train)

注意，X_train既用来做输入，也用来做目标。图 17-2 显示了原始 3D 数据集（左侧）和自编码器隐藏层的输出（即编码层，右侧）。 可以看到，自编码器找到了投影数据的最佳二维平面，保留了数据的尽可能多的差异（就像 PCA 一样）。

<img src='
https://hands1ml.apachecn.org/img/e6301e8ec35adb6618df0c051f883dce.png
'>
<br>

笔记：可以将自编码器当做某种形式的自监督学习（带有自动生成标签功能的监督学习，这个例子中标签等于输入）

### 栈式自编码器
就像我们讨论过的其他神经网络一样，自编码器可以有多个隐藏层。 在这种情况下，它们被称为栈式自编码器（或深度自编码器）。 添加更多层有助于自编码器了解更复杂的编码。 但是，必须注意不要让自编码器功能太强大。 设想一个编码器非常强大，只需学习将每个输入映射到一个任意数字（并且解码器学习反向映射）即可。 很明显，这样的自编码器将完美地重构训练数据，但它不会在过程中学习到任何有用的数据表征（并且它不可能很好地泛化到新的实例）。

栈式自编码器的架构以中央隐藏层（编码层）为中心通常是对称的。 简单来说，它看起来像一个三明治。 例如，一个用于 MNIST 的自编码器（在第 3 章中介绍）可能有 784 个输入，其次是一个隐藏层，有 100 个神经元，然后是一个中央隐藏层，有 30 个神经元，然后是另一个隐藏层，有 100 个神经元，输出层有 784 个神经元。 这个栈式自编码器如图 17-3 所示。

<img src='
https://hands1ml.apachecn.org/img/2eb643c0821b45d0728233dbf25d1e46.png
'>
<br>


#### 用 Keras 实现栈式自编码器
你可以像常规深度 MLP 一样实现栈式自编码器。 特别是，我们在第 11 章中用于训练深度网络的技术也可以应用。例如，下面的代码使用 SELU 激活函数为 Fashion MNIST 创建了一个栈式自编码器：

In [ ]:
stacked_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(30, activation="selu"),
])
stacked_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])
stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])
stacked_ae.compile(loss="binary_crossentropy",
                   optimizer=keras.optimizers.SGD(lr=1.5))
history = stacked_ae.fit(X_train, X_train, epochs=10,
                         validation_data=[X_valid, X_valid])

逐行看下这个代码：

和之前一样，自编码器包括两个子模块：编码器和解码器。

编码器接收28 × 28像素的灰度图片，打平为大小等于 784 的向量，用两个紧密层来处理，两个紧密层都是用 SELU 激活函数（还可以加上 LeCun 归一初始化，但因为网络不深，效果不大）。对于每张输入图片，编码器输出的向量大小是 30。

解码器接收大小等于 30 的编码（编码器的输出），用两个紧密层来处理，最后的向量转换为28 × 28的数组，使解码器的输出和编码器的输入形状相同。

编译时，使用二元交叉熵损失，而不是 MSE。将重建任务当做多标签分类问题：每个像素强度表示像素应该为黑色的概率。这么界定问题（而不是当做回归问题），可以使模型收敛更快。

最后，使用X_train既作为输入，也作为目标，来训练模型（相似的，使用X_valid既作为验证的输入也作为目标）。

#### 可视化重建
确保自编码器训练得当的方式之一，是比较输入和输出：差异不应过大。画一些验证集的图片，及其重建：

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap="binary")
    plt.axis("off")

def show_reconstructions(model, n_images=5):
    reconstructions = model.predict(X_valid[:n_images])
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(X_valid[image_index])
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index])

show_reconstructions(stacked_ae)

可以认出重建，但图片有些失真。需要再训练模型一段时间，或使编码器和解码器更深，或使编码更大。但如果使网络太强大，就学不到数据中的规律。

#### 可视化 Fashion MNIST 数据集
训练好栈式自编码器之后，就可以用它给数据集降维了。可视化的话，结果不像（第 8 章其它介绍的）其它降维方法那么好，但自编码器的优势是可以处理带有多个实例多个特征的大数据集。所以一个策略是利用自编码器将数据集降维到一个合理的水平，然后使用另外一个降维算法做可视化。用这个策略来可视化 Fashion MNIST。首先，使用栈式自编码器的编码器将维度降到 30，然后使用 Scikit-Learn 的 t-SNE 算法实现，将维度降到 2 并做可视化：

In [ ]:
from sklearn.manifold import TSNE

X_valid_compressed = stacked_encoder.predict(X_valid)
tsne = TSNE()
X_valid_2D = tsne.fit_transform(X_valid_compressed)

# 对数据集作图：
plt.scatter(X_valid_2D[:, 0], X_valid_2D[:, 1], c=y_valid, s=10, cmap="tab10") 

图 17-5 展示了结果的散点图（并展示了一些图片）。t-SNE 算法区分除了几类，比较符合图片的类别（每个类的颜色不一样）。

<img src='
https://hands1ml.apachecn.org/img/fc72779301071ddc44cc9423b21732bc.png
'>
<br>

自编码器的另一个用途是无监督预训练。

#### 使用栈式自编码器做无监督预训练
第 11 章讨论过，如果要处理一个复杂的监督任务，但又缺少标签数据，解决的方法之一，是找一个做相似任务的神经网络，复用它的底层。这么做就可以使用少量训练数据训练出高性能的模型，因为模型不必学习所有低层次特征；模型可以复用之前的特征探测器。

相似的，如果有一个大数据集，但大部分实例是无标签的，可以用全部数据训练一个栈式自编码器，然后使用其底层创建一个神经网络，再用有标签数据来训练。例如，图 17-6 展示了如何使用栈式自编码器来做分类的无监督预训练。当训练分类器时，如果标签数据不足，可以冻住预训练层（底层）。

<img src='
https://hands1ml.apachecn.org/img/6a262b3ada6d315c0ff9d176785f0e0d.png
'>
<br>


笔记：无标签数据很多，有标签数据数据很少，非常普遍。搭建一个大无便签数据集很便宜（比如，一段小脚本可以从网上下载许多图片），但是给这些图片打标签（比如，将其标签为可爱或不可爱）只有人做才靠谱。打标签又耗时又耗钱，所以人工标注实例有几千就不错了。

代码实现没有特殊之处：用所有训练数据训练自编码器，然后用编码器层创建新的神经网络（本章有练习题例子）。

接下来，看看关联权重的方法。

#### 关联权重
当自编码器整齐地对称时，就像我们刚刚构建的那样，一种常用方法是将解码器层的权重与编码器层的权重相关联。 这样减半了模型中的权重数量，加快了训练速度，并限制了过度拟合的风险。具体来说，如果自编码器总共具有N个层（不算输入层），并且W[L]表示第L层的连接权重（例如，层 1 是第一隐藏层，则层N / 2是编码层，而层N是输出层），则解码器层权重可以简单地定义为：W[N–L+1] = W[L]^T（其中L = 1, 2, ..., N/2）。

使用 Keras 将层的权重关联起来，先定义一个自定义层：

In [ ]:
class DenseTranspose(keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias", initializer="zeros",
                                      shape=[self.dense.input_shape[-1]])
        super().build(batch_input_shape)
    def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)
        return self.activation(z + self.biases)

自定义层的作用就像一个常规紧密层，但使用了另一个紧密层的权重，并且做了转置（设置transpose_b=True等同于转置第二个参数，但在matmul()运算中实时做转置更为高效）。但是，要使用自己的偏置向量。然后，创建一个新的栈式自编码器，将解码器的紧密层和编码器的紧密层关联起来：

In [ ]:
dense_1 = keras.layers.Dense(100, activation="selu")
dense_2 = keras.layers.Dense(30, activation="selu")

tied_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    dense_1,
    dense_2
])

tied_decoder = keras.models.Sequential([
    DenseTranspose(dense_2, activation="selu"),
    DenseTranspose(dense_1, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

tied_ae = keras.models.Sequential([tied_encoder, tied_decoder])

这个模型的重建误差小于前一个模型，且参数量只有一半。

#### 一次训练一个自编码器
不是一次完成整个栈式自编码器的训练，而是一次训练一个浅自编码器，然后将所有这些自编码器堆叠到一个栈式自编码器（因此名称）中，通常要快得多，如图 17-7 所示。 这个方法如今用的不多了，但偶尔还会撞见谈到“贪婪层级训练”的论文，所以还是看一看。

<img src='
https://hands1ml.apachecn.org/img/2e97115a3976ddb176e401dcfa95b53c.png
'>
<br>


在训练的第一阶段，第一个自编码器学习重构输入。 然后，使用整个训练集训练第一个自编码器，得到一个新的（压缩过的）训练集。然后用这个数据集训练第二个自编码器。这是第二阶段的训练。最后，我们用所有这些自编码器创建一个三明治结构，见图 17-7（即，先把每个自编码器的隐藏层叠起来，再加上输出层）。这样就得到了最终的栈式自编码器（见笔记本）。我们可以用这种方式训练更多的自编码器，搭建非常深的栈式自编码器。

正如前面讨论过的，现在的一大趋势是 Geoffrey Hinton 等人在 2006 年发现的，靠这种贪婪层级方法，可以用无监督方式训练神经网络。他们还使用了受限玻尔兹曼机（RBM，见附录 E）。但在 2007 年，Yoshua Bengio 发现只用自编码器也可以达到不错的效果。在这几年间，自编码器是唯一的有效训练深度网络的方法，知道出现第 11 章介绍过的方法。

自编码器不限于紧密网络：还有卷积自编码器和循环自编码器。

### 卷积自编码器
如果处理的是图片，则前面介绍的自编码器的效果可能一般（除非图片非常小）。第 14 章介绍过，对于图片任务，卷积神经网络比紧密网络的效果更好。所以如果想用自编码器来处理图片的话（例如，无监督预训练或降维），你需要搭建一个卷积自编码器。编码器是一个包含卷积层和池化层的常规 CNN。通常降低输入的空间维度（即，高和宽），同时增加深度（即，特征映射的数量）。解码器的工作相反（放大图片，压缩深度），要这么做的话，可以转置卷积层（或者，可以将上采样层和卷积层合并）。下面是一个卷积自编码器处理 Fashion MNIST 的例子：

In [ ]:
conv_encoder = keras.models.Sequential([
    keras.layers.Reshape([28, 28, 1], input_shape=[28, 28]),
    keras.layers.Conv2D(16, kernel_size=3, padding="same", activation="selu"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="selu"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="selu"),
    keras.layers.MaxPool2D(pool_size=2)
])
conv_decoder = keras.models.Sequential([
    keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2, padding="valid",
                                 activation="selu",
                                 input_shape=[3, 3, 64]),
    keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, padding="same",
                                 activation="selu"),
    keras.layers.Conv2DTranspose(1, kernel_size=3, strides=2, padding="same",
                                 activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])
conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])